# Retrieving All Usefull Unscaled Bands of the GEE Landsat 5 Image Collection 

## Import Packages and Initialize GEE

In [23]:
##Import required Google Earth Engine python packages and check if they work in python environment
import ee
#ee.Authenticate(force=True) # Uncomment this if it is first time with GEE or account linking is having issues
ee.Initialize()
import geetools
import geemap
import os
import time

In [24]:
#import the map module that allows for mapping images to an interactive map
Map = geemap.Map()

## Import Google Earth Engine Feature Collection Data

In [25]:
#Import the river boundary from the Google Earth Engine Server
#Call in river in from a vector file saved into Google Earth Engine
TN_River = ee.FeatureCollection("users/pjf927/TN_River_GERS_StudySite") # You will have to download your own file (here: https://liveutk-my.sharepoint.com/:u:/r/personal/hrr794_utc_edu/Documents/GERS_Lab_Research/Data/River_Boundaries/TN_River_GERS_StudySite_UTM_Zone_16_Northern_Hemisphere.zip?csf=1&web=1&e=ZtPR3f). shp files can be upload by: https://youtu.be/_cR_ORXyo78?si=0-XDIc0OpOo_YCFi 
#Some function require geometry values to clip features
TN_RiverGeom = TN_River.geometry() 
#Generate a square boundary around the river study area
RiverBounds = TN_RiverGeom.bounds()

## Import Google Earth Engine Image Collection Data

In [26]:
#Call in Landsat 5 Level 2, Collection 2, Tier 1 dataset. Info found at: https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LT05_C02_T1_L2
LS5_SR = (
    ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")
    .filterBounds(TN_River) #Filter only swath grids that cover the TN River Boundary
    .select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4','SR_B5', 'SR_B7', 'ST_B6', 'QA_PIXEL', 'QA_RADSAT'],['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2', 'Kelvin', 'QA_PIXEL', 'QA_RADSAT']) #Assign imagery bands new names
    .filterDate('1984-03-15', '2012-05-06') # First date: 1984-03-16  Last date: 2012-05-05. Recomended to process only 5 year intervals due to length of the data
    .sort('system:time_start') #Sort collection by acquisition time
)

#Get a count of all images filtered in the Landsat Surface Relectance Collection
LS5_count_raw = LS5_SR.size().getInfo()
print("Landsat 5 Images: ", LS5_count_raw)

Landsat 5 Images:  1831


## Function of mosaicing images of the same date together 

In [5]:
#Function declaration to later use on interating through an image collection
def mergeByDate(imgCol):
    #Convert the image collection to a list.
    imgList = imgCol.toList(imgCol.size())
    
    # Driver function for mapping the unique dates
    def uniqueDriver(image):
        return ee.Image(image).date().format("YYYY-MM-dd")
    
    uniqueDates = imgList.map(uniqueDriver).distinct()

    # Driver function for mapping the moasiacs
    def mosaicDriver(date):
        date = ee.Date(date)
        
        image = (imgCol
               .filterDate(date, date.advance(1, "day"))
               .mosaic())
        
        return image.set(
                        "system:time_start", date.millis(), 
                        "system:id", date.format("YYYY-MM-dd"),
                        "Date", date.format("YYYY-MM-dd"))
    
    mosaicImgList = uniqueDates.map(mosaicDriver)
    
    return ee.ImageCollection(mosaicImgList)

## Run the mosaic by date function on the image collection

In [6]:
LS5_SR_Mosaic = mergeByDate(LS5_SR)

## Function of converting the QA band bit formated data to a class of pixel (flag bit)

In [7]:
# Function declaration for classifying the QA band. QA band info can be found here on page 13: https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/media/files/LSDS-1619_Landsat8-9-Collection2-Level2-Science-Product-Guide-v6.pdf
def getQABits(image, start, end, newName):
    # Compute the bits we need to extract.
    pattern = 0
    for i in range(start, end+1):
       pattern += pow(2, i)
    # Return a single band image of the extracted QA bits, giving the band a new name.
    return image.select([0], [newName]).bitwiseAnd(pattern).rightShift(start)

## Function of of determining the percentage of clouds within the water body boundary feature class

In [8]:
# Function declaration to calculate the percentage of clouds within the water body boundary feature class
def cloud_mask(image):
    # Select the QA band.
    QA = image.select(['QA_PIXEL'])
    # Get the internal_cloud_algorithm_flag bit.
    cloud_only = getQABits(QA, 3,3, 'cloud_mask').eq(1)
    cloudiness = cloud_only.reduceRegion(
             reducer = ee.Reducer.mean(), 
             geometry = TN_RiverGeom, 
             scale = 30,
             maxPixels = 1e13).get("cloud_mask")
    percent = ee.Number(cloudiness).multiply(100)
    return image.set("CLOUD_COVER", percent)

## Run the cloud percentage function on the image collection

In [9]:
LS5_SR_Mask = LS5_SR_Mosaic.map(cloud_mask)

## Change all dictionary values in 'SPACECRAFT_ID' to 'LANDSAT_4'

In [10]:
LS5_SR_Sensor = LS5_SR_Mask.map(lambda image: image.set('SPACECRAFT_ID', 'LANDSAT_5'))

## Convert the acqusition date time to a 'YYYY-MM-dd' format

In [11]:
LS5_SR_Sensor = LS5_SR_Sensor.map(
    lambda img: img.set({"Date": ee.Date(img.get("system:time_start")).format("YYYY-MM-dd")})
)

## Create a new image collection of images that are filtered out based on less than 30% cloud cover using the new 'CLOUD_COVER' percentage dictionary item

In [12]:
LS5_SR_Filtered = LS5_SR_Sensor.filterMetadata("CLOUD_COVER","less_than", 30)

## Get a count of all images in the image collection

In [22]:
# Every time the .getInfo() function is used, processing time goes up. It's a good idea to use a start and endtime on the first .getInfo() to gauge how long the others will be
start = time.time()
All_count_preprocess = LS5_SR_Filtered.size().getInfo()
print("Landsat Images: ", All_count_preprocess)
end = time.time()
print("Processing Time = " + str(round((end - start)/60, 2)) + " Minutes") # time in Minutes

Landsat Images:  95
Processing Time = 1.18 Minutes


## Get all the dates of the image collection form the new 'Date' dictionary item

In [14]:
dates = LS5_SR_Filtered.aggregate_array("Date").getInfo()
print("Dates in Imagecollection: ", dates)
#Print out the first and last dates of the created list
first = dates[0]
last = dates[-1]
print("The image collection is from " + first + " to " + last)

Dates in Imagecollection:  ['1999-05-29', '1999-06-21', '1999-07-07', '1999-07-16', '1999-07-23', '1999-08-01', '1999-08-17', '1999-09-02', '1999-09-25', '1999-10-27', '1999-11-05', '1999-11-12', '1999-11-28', '1999-12-07', '1999-12-23', '1999-12-30', '2000-01-15', '2000-03-12', '2000-03-28', '2000-04-29', '2000-05-06', '2000-05-15', '2000-05-22', '2000-05-31', '2000-06-07', '2000-06-23', '2000-07-02', '2000-07-09', '2000-08-03', '2000-08-19', '2000-09-04', '2000-09-11', '2000-09-27', '2000-10-13', '2000-10-22', '2000-11-30', '2000-12-09', '2000-12-25', '2001-01-10', '2001-01-26', '2001-02-18', '2001-03-06', '2001-03-22', '2001-04-16', '2001-04-23', '2001-05-02', '2001-05-18', '2001-06-10', '2001-06-19', '2001-06-26', '2001-08-22', '2001-09-14', '2001-09-23', '2001-09-30', '2001-10-09', '2001-10-25', '2001-11-01', '2001-11-10', '2001-11-17', '2001-11-26', '2001-12-03', '2001-12-19', '2001-12-28', '2002-01-04', '2002-01-13', '2002-02-14', '2002-02-21', '2002-06-22', '2002-07-08', '2002-

## Get all the names of the space crafts in the image collection

In [18]:
sensor = LS5_SR_Filtered.aggregate_array("SPACECRAFT_ID").getInfo()
print("sensor: ", sensor)

sensor:  ['LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'LANDSAT_5', 'L

## Clip out the entire image to the dimensions of the water body boundary feature class

In [15]:
#Uncommit the last line of code if you just want the pixels within that boundary. Having the whole image is useful to see where clouds are.
#Clip out all pixels that are IN the TN River boundary and save as a new Image Collection using the lambda function
#LS5_SR_Clip = LS5_SR_Filtered.map(lambda image: image.clip(TN_River))

## Create a list and cound the number of images to be later used to export the whole filetered image collection

In [19]:
LS5_SR_Filtered_List = LS5_SR_Filtered.toList(LS5_SR_Filtered.size())

## Create a for loop that exports all images to a folder in your local drive or your Google drive

In [20]:
#True color iterate images to add map to layer, export to local directory, and export to Google Drive
for index in range(0, All_count_preprocess):
    image = ee.Image(LS5_SR_Filtered_List.get(index))
    layer_name = "All_Bands_NonScaled_" + str(dates[index]) + "_" + str(sensor[index])
    #Map.addLayer(image, trueVis, layer_name, False) #Uncomment this line if you would like to see the image in the map window of the GEE map interactive
    #filename = os.path.join(r'D:\Thesis\ASPRS\GEE_Assessment\Data\Rasters\True_Test', layer_name + ".tif") #Uncomment this line if you would like to create a path to a local folder on your computer to export
    #geemap.ee_export_image(image, filename = filename, scale = 30, region = TN_RiverGeom, file_per_band = False) #Uncomment this line if you would like to export the data localy to the declared folder. Note: Big image exports won't work
    geemap.ee_export_image_to_drive(image, description = layer_name, folder = 'LS5_' + first + '_to_' + last, region = TN_RiverGeom, scale = 30)